This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [34]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'wyuWV4cyusRe1iru7amh'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import statistics
import json

In [36]:
print ("""https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-04-15
                             &end_date=2017-04-15&api_key="""+API_KEY)

https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-04-15
                             &end_date=2017-04-15&api_key=wyuWV4cyusRe1iru7amh


In [37]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
quand_request = requests.get( """https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-07-02
                             &end_date=2017-07-03&api_key=""" + API_KEY)

In [38]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print (quand_request.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-07-02', 'end_date': '2017-07-03', 'frequency': 'daily', 'data': [['2017-07-03', 45.29, 45.83, 45.06, 45.75, None, 71381.0, 3251502.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [39]:
year_request = requests.get ("""https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01
                             &end_date=2017-12-31&api_key=""" + API_KEY)

### 2. Convert the returned JSON object into a Python dictionary.

In [40]:

# Returned as a python dictionary# Return 
year_dict = year_request.json()
print (type(year_dict))

<class 'dict'>


### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [52]:
open_dict = {row[0]: row[1] for row in year_dict["dataset_data"]["data"] if row[1] != None}
open_max_date, open_max = max(open_dict.items(), key=lambda k: k[1])
open_min_date, open_min = min(open_dict.items(), key=lambda k: k[1])

print("The highest opening price in 2017 was ${0:.2f} per share and occured on {1}." \
      .format(round(open_max, 2), open_max_date))
print("The lowest opening price in 2017 was ${0:.2f} per share and oocurred on {1}." \
      .format(round(open_min, 2), open_min_date))

The highest opening price in 2017 was $53.11 per share and occured on 2017-12-14.
The lowest opening price in 2017 was $34.00 per share and oocurred on 2017-01-24.


### 4. What was the largest change in any one day (based on High and Low price)?

In [55]:
#High occurs at [2] and low occurs at [3]
'''
max_change = 0
for day in data:
    if (day[2] - day[3]) > max_change:
        max_change = (day[2] - day[3])
        
print ('Highest change price:', max_change)
'''
spread_dict = {row[0]: row[2] - row[3] for row in year_dict["dataset_data"]["data"]}
spread_max_date, spread_max = max(spread_dict.items(), key=lambda k: k[1])

print("The largest daily price spread in 2017 was {1}${0:.2f} per share and occurred on {2}." \
      .format(round(spread_max, 2), u'\u0394', spread_max_date))

The largest daily price spread in 2017 was Δ$2.81 per share and occurred on 2017-05-11.


### 5. What was the largest change between any two days (based on Closing Price)?

In [58]:
#Largest price change occurs at [4]
largest_two_days = 0
for day in range(0, len(data)-1):
    two_days = abs(data[day][4] - data[day+1][4])
    if two_days > largest_two_days:
        largest_two_days = two_days
print ('Largest price change between two days:', round(largest_two_days,2))

Largest price change between two days: 2.56


### 6. What was the average daily trading volume during this year?

In [59]:
#Trading volume occurs at [6]
#Using the statistics library
average_trading = statistics.mean([day[6] for day in data])
print ('Average daily trading volume for 2017:', round(average_trading,2))

Average daily trading volume for 2017: 89124.34


### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [47]:

#Using the statisics library#Using t 
median_trading_volume = statistics.median([day[6] for day in data])
print ('Median trading volume for 2017:', median_trading_volume)

Median trading volume for 2017: 76286.0
